In [2]:
import numpy as np
import matplotlib.pyplot as plt
import xtrack as xt
import xcoll as xc

In [3]:
def install_tidp(line, block_mvt=29e-3):
    tidp_ap_tot = 147e-3
    line.discard_tracker()
    tidp = xc.EverestCollimator(length=4.3, material=xc.materials.Carbon, jaw_L= tidp_ap_tot/2 + block_mvt, jaw_R = -tidp_ap_tot/2 + block_mvt)
    line.collimators.install(names=['tidp.11434'], elements=[tidp])
    return tidp

def install_tcsm(line):
    tcsm = xc.EverestCollimator(length=1.83, gap=5, material=xc.materials.Carbon) # length is 1.83
    line.collimators.install(names=['tcsm.51932'], elements=[tcsm])
    return tcsm

def install_offmom_bpms_colls(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2):
    tw = line.twiss()
    tt = line.get_table()
    mask_disp = 5*np.sqrt(tw.betx*exn*pmass/nrj)+n_buckets*bucket_height*tw.dx > 0.025
    mask_bpm = ['bp' in name for name in tt.name]
    mask_aper = np.array(['aper' in name for name in tt.name])
    offmom_bpms = tt.name[mask_disp & mask_bpm & ~mask_aper]
    colls = []
    aper_to_remove = []
    for nn in offmom_bpms:
        aper_to_remove.append(f'{nn}.a_aper')
        aper_to_remove.append(f'{nn}.b_aper')
        if line[nn+'.a_aper'].__class__.__name__ == 'LimitEllipse':
            jaw = line[nn+'.a_aper'].a
        else:
            jaw = line[nn+'.a_aper'].max_x
        
        colls.append(xc.EverestCollimator(length=line[nn].length, material=xc.materials.Beryllium, jaw=jaw))
    line.remove(aper_to_remove)
    line.collimators.install(names=offmom_bpms, elements=colls)
    return colls

def remove_offmom_bpms_apers(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2):
    "Remove apertures of off-momentum BPMs which give flanges as bottlenecks"
    tw = line.twiss()
    tt = line.get_table()
    mask_disp = 5*np.sqrt(tw.betx*exn*pmass/nrj)+n_buckets*bucket_height*tw.dx > 0.025
    mask_bpm = ['bp' in name for name in tt.name]
    mask_aper = np.array(['aper' in name for name in tt.name])
    offmom_bpms = tt.name[mask_disp & mask_bpm & ~mask_aper]
    aper_to_remove = [f'{name}{suffix}' for name in offmom_bpms for suffix in ('.a_aper', '.b_aper')]
    line.remove(aper_to_remove)

In [4]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta4.json')
tt1 = line.get_table()
remove_offmom_bpms_apers(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2)
tt = line.get_table()
# tw = line.twiss()

#shift VEB apertures by 5.3 mm to account for the sagitta
veb_b_apers = tt.rows['veb.*.b_aper'].name
for name in veb_b_apers:
    line[name].shift_x += 5.3e-3

veb_a_apers = tt.rows['veb.*.a_aper'].name
for name in veb_a_apers:
    line[name].shift_x += 5.3e-3
#Houdt de a aperture steek?
tt = line.get_table()
tw = line.twiss()

Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


In [5]:
L1 = 34e-3
L2 = 70e-3

r1 = 41.5e-3
r2 = 60e-3-1.5e-3
r3 = 51.5e-3

changed = [10110, 11110, 11310, 12510, 13510, 20910, 21110, 22510, 23510, 30110, 30910,
32510, 33510, 40110, 40910, 41110, 42510, 42710, 51110, 52510, 53510, 60110,
61110, 62510, 63510]

In [ ]:
def slice_line_for_pipe(line, apertures, pipe_name, pipe_positions=None):
    tt = line.get_table()

    # Placing start and end aperture of the pipe if not already present
    if pipe_positions is None:
        if f'{pipe_name}.a_aper' in tt.name and f'{pipe_name}.b_aper' in tt.name:
            pipe_start = f'{pipe_name}.a_aper'
            pipe_end = f'{pipe_name}.b_aper'
        else:
            raise ValueError(f'Pipe apertures not found in line for {pipe_name}, please provide pipe_positions')
    else:
        pipe_start = None
        pipe_end = None
        for nn in tt.rows[(tt.s > pipe_positions[0] - 1e-3 ) & (tt.s < pipe_positions[0] + 1e-2 )].name:
            if line[nn].__class__.__name__.startswith('Limit'):
                pipe_start = nn

        for nn in tt.rows[(tt.s > pipe_positions[-1] - 1e-3 ) & (tt.s < pipe_positions[-1] + 1e-2 )].name:
            if line[nn].__class__.__name__.startswith('Limit'):
                pipe_end = nn
                break
        if pipe_start is None or pipe_end is None:
            pipe_start = f'{pipe_name}.a_aper'
            pipe_end = f'{pipe_name}.b_aper'
            line.env.elements[pipe_start] = apertures[0].copy()
            line.env.elements[pipe_end] = apertures[-1].copy()
            line.insert([line.env.place(pipe_start, at=pipe_positions[0]),
                         line.env.place(pipe_end, at=pipe_positions[-1])], s_tol=1e-6)
            
    
    # Finding active elements in the pipe
    tt = line.get_table() #update table to find newly placed apertures
    active_elements = []
    for nn, ee in zip(tt.rows[f'{pipe_start}>>1':pipe_end].name, tt.rows[f'{pipe_start}>>1':pipe_end].element_type):
        if ee.startswith('Drift') or ee.startswith('Marker') or ee.startswith('Limit'):
            continue
        else:
            active_elements.append(nn)
    if len(active_elements) == 0:
        print(f'No active elements in pipe {pipe_name}, no need to slice')
        return None
    else:
        print(f'Active elements in pipe {pipe_name}: {active_elements}')

    # Slicing strategies
    slicing_strategies = [xt.Strategy(slicing=None)]
    for nn in active_elements:
        slicing_strategies.append(xt.Strategy(slicing=xt.Teapot(5), name=nn))
    line.slice_thick_elements(slicing_strategies)

In [ ]:
def get_slicing_strategies():
    pass

In [24]:
def find_pipe_edges(line, pipe_data):
    tt = line.get_table()

    #pipe names
    pipe_names = list(pipe_data.keys())
    apers_to_add = {}
    
    for pipe_name in pipe_names:
        if 'edge_names' in pipe_data[pipe_name]:
            assert pipe_data[pipe_name]['edge_names'][0] in tt.name and pipe_data[pipe_name]['edge_names'][1] in tt.name, f"Pipe edge names {pipe_data[pipe_name]['edge_names']} not found in line for {pipe_name}"
        
        else:
            assert 'edge_positions' in pipe_data[pipe_name], f"Please provide edge names or edge positions for {pipe_name}"

            #Looking if apertures already exist in line
            pipe_start = None
            pipe_end = None
            for nn in tt.rows[(tt.s > pipe_data[pipe_name]['edge_positions'][0] - 1e-3 ) & (tt.s < pipe_data[pipe_name]['edge_positions'][0] + 1e-2 )].name:
                if line[nn].__class__.__name__.startswith('Limit'):
                    pipe_start = nn

            for nn in tt.rows[(tt.s > pipe_data[pipe_name]['edge_positions'][-1] - 1e-3 ) & (tt.s < pipe_data[pipe_name]['edge_positions'][-1] + 1e-2 )].name:
                if line[nn].__class__.__name__.startswith('Limit'):
                    pipe_end = nn
                    break
            
            if pipe_start is None or pipe_end is None:
                apers_to_add[pipe_name] = {'positions': pipe_data[pipe_name]['edge_positions'],
                                           'names': [f'{pipe_name}.a_aper', f'{pipe_name}.b_aper']}
                pipe_data[pipe_name]['edge_names'] = [f'{pipe_name}.a_aper', f'{pipe_name}.b_aper']
            else:
                pipe_data[pipe_name]['edge_names'] = [pipe_start, pipe_end]

    if len(list(apers_to_add.keys()))>0:
        insertions = []
        for pipe_name in apers_to_add.keys():
            line.env.elements[apers_to_add[pipe_name]['names'][0]] = pipe_data[pipe_name]['apertures'][0].copy()
            line.env.elements[apers_to_add[pipe_name]['names'][1]] = pipe_data[pipe_name]['apertures'][-1].copy()

        insertions += [line.env.place(apers_to_add[pipe_name]['names'][0], at=apers_to_add[pipe_name]['positions'][0]),
                        line.env.place(apers_to_add[pipe_name]['names'][1], at=apers_to_add[pipe_name]['positions'][1])]

        line.insert(insertions, s_tol=1e-6)

    return pipe_data

In [25]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta4.json')
tt1 = line.get_table()
remove_offmom_bpms_apers(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2)
tt = line.get_table()
# tw = line.twiss()

#shift VEB apertures by 5.3 mm to account for the sagitta
veb_b_apers = tt.rows['veb.*.b_aper'].name
for name in veb_b_apers:
    line[name].shift_x += 5.3e-3

veb_a_apers = tt.rows['veb.*.a_aper'].name
for name in veb_a_apers:
    line[name].shift_x += 5.3e-3
#Houdt de a aperture steek?
tt = line.get_table()
tw = line.twiss()

Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


In [27]:
pipe_data = {}

for nn in changed:
    flange_number = nn-9
    pipe_name = f'vcak.{flange_number}'
    
    L3 = line.get_s_position(f'vcak.{flange_number}.b_aper') - line.get_s_position(f'vcak.{flange_number}.a_aper') - L1 - L2
    if L3 < 0:
        raise ValueError(f'Pipe {pipe_name} too short to fit all apertures, please adjust L1 and L2')
    
    pipe_data[pipe_name] = {'apertures': [xt.LimitEllipse(a=r1, b=r1, shift_x=-5.3e-3), 
                                  xt.LimitEllipse(a=r2, b=r2, shift_x=-5.3e-3),
                                  xt.LimitEllipse(a=r3, b=r3)],
                            'lengths': [L1, L2, L3],
                            #'edge_names': [f'{pipe_name}.a_aper', f'{pipe_name}.b_aper'],
                            'edge_positions': [line.get_s_position(f'vcak.{flange_number}.a_aper'), line.get_s_position(f'vcak.{flange_number}.b_aper')]
                            }

In [28]:
pipe_data = find_pipe_edges(line, pipe_data)

In [29]:
tt = line.get_table()

Both methods seem to work (if edge names are given, obv it is ok, if I give positions without names, it assigns the right name to it)

In [36]:
def get_slicing_strategies(line, pipe_data):
    slicing_strategies = [xt.Strategy(slicing=None)]
    pipe_names = list(pipe_data.keys())
    tt = line.get_table()

    active_elements = []
    # Finding active elements in the pipe
    for pipe_name in pipe_names:
        pipe_start, pipe_end = pipe_data[pipe_name]['edge_names']
        for nn, ee in zip(tt.rows[f'{pipe_start}>>1':pipe_end].name, tt.rows[f'{pipe_start}>>1':pipe_end].element_type):
            if ee.startswith('Drift') or ee.startswith('Marker') or ee.startswith('Limit'):
                continue
            else:
                active_elements.append(nn)
    
    for nn in active_elements:
        slicing_strategies.append(xt.Strategy(slicing=xt.Teapot(5), name=nn))
    return slicing_strategies

In [39]:
slicing_strategies = get_slicing_strategies(line, pipe_data)
line.slice_thick_elements(slicing_strategies)

Slicing line:   0%|          | 0/32246 [00:00<?, ?it/s]

{'sps$start': ['sps$start'],
 'begi.10010': ['begi.10010'],
 'qf.10010': ['qf.10010'],
 'drift_0..0': ['drift_0..0'],
 'veqf.10010.b_aper': ['veqf.10010.b_aper'],
 'drift_0..1': ['drift_0..1'],
 'veba.10030.a_aper': ['veba.10030.a_aper'],
 'drift_0..2': ['drift_0..2'],
 'mba.10030_entry': ['mba.10030_entry'],
 'mba.10030..entry_map': ['mba.10030..entry_map'],
 'mba.10030..0_aper2': ['mba.10030..0_aper2'],
 'mba.10030..0': ['mba.10030..0'],
 'mba.10030..1_aper2': ['mba.10030..1_aper2'],
 'mba.10030..1': ['mba.10030..1'],
 'mba.10030..2_aper2': ['mba.10030..2_aper2'],
 'mba.10030..2': ['mba.10030..2'],
 'mba.10030..3_aper2': ['mba.10030..3_aper2'],
 'mba.10030..3': ['mba.10030..3'],
 'mba.10030..4_aper2': ['mba.10030..4_aper2'],
 'mba.10030..4': ['mba.10030..4'],
 'mba.10030..5_aper2': ['mba.10030..5_aper2'],
 'mba.10030..5': ['mba.10030..5'],
 'mba.10030..6_aper2': ['mba.10030..6_aper2'],
 'mba.10030..6': ['mba.10030..6'],
 'mba.10030..7_aper2': ['mba.10030..7_aper2'],
 'mba.10030..7': 

In [40]:
tt = line.get_table()

In [48]:
tt.rows['vcak.13501.a_aper':'vcak.13501.b_aper']

Table: 19 rows, 11 cols
name                             s element_type           isthick isreplica parent_name ...
vcak.13501.a_aper          1117.77 LimitEllipse             False     False None       
drift_299..2               1117.77 DriftSlice                True     False drift_299  
lod.13502_entry            1117.88 Marker                   False     False None       
lod.13502..entry_map       1117.88 ThinSliceOctupoleEntry   False     False lod.13502  
drift_lod.13502..0         1117.88 DriftSliceOctupole        True     False lod.13502  
lod.13502..0               1117.93 ThinSliceOctupole        False     False lod.13502  
drift_lod.13502..1         1117.93 DriftSliceOctupole        True     False lod.13502  
lod.13502..1               1118.07 ThinSliceOctupole        False     False lod.13502  
drift_lod.13502..2         1118.07 DriftSliceOctupole        True     False lod.13502  
lod.13502..2               1118.21 ThinSliceOctupole        False     False lod.13502  
drif

In [58]:
line.get_s_position('vcak.13501.a_aper') + L1

np.float64(1117.8072000000002)

In [70]:
ll = list(tt.rows[(tt.s>=line.get_s_position('vcak.13501.a_aper')) & (tt.s<=line.get_s_position('vcak.13501.a_aper')+L1)].name)
if ll[0].startswith('vcak'):
    ll.pop(0)

In [73]:
names = list(tt.rows[(tt.s>=line.get_s_position('vcak.13501.a_aper')) & (tt.s<=line.get_s_position('vcak.13501.a_aper')+L1)].name)
elements = list(tt.rows[(tt.s>=line.get_s_position('vcak.13501.a_aper')) & (tt.s<=line.get_s_position('vcak.13501.a_aper')+L1)].element_type)

if 'vcak' in names[0]:
    names.pop(0)
    elements.pop(0)

In [76]:
for nn, ee in zip(names, elements):
    print(nn, ee)

drift_299..2 DriftSlice


In [53]:
tt.rows['vcak.13501.a_aper'].s[0]+L1

np.float64(1117.8072000000002)

In [ ]:
def install_collimators(line, pipe_data):
    tt = line.get_table()
    pipe_names = list(pipe_data.keys())
    collimators = []

    for pipe_name in pipe_names:
        pipe_start, pipe_end = pipe_data[pipe_name]['edge_names']
        for nn in tt.rows[f'{pipe_start}>>1':pipe_end].name:
            

In [146]:
line = xt.Line.from_json('../injection_lines/sps_with_aperture_inj_q20_beam_sagitta4.json')
tt1 = line.get_table()
remove_offmom_bpms_apers(line, exn=3.5e-6, nrj=21, pmass=0.938, bucket_height=3e-3, n_buckets=2)
tt = line.get_table()
# tw = line.twiss()

#shift VEB apertures by 5.3 mm to account for the sagitta
veb_b_apers = tt.rows['veb.*.b_aper'].name
for name in veb_b_apers:
    line[name].shift_x += 5.3e-3

veb_a_apers = tt.rows['veb.*.a_aper'].name
for name in veb_a_apers:
    line[name].shift_x += 5.3e-3
#Houdt de a aperture steek?
tt = line.get_table()
tw = line.twiss()


pipe_data = {}

for nn in changed:
    flange_number = nn-9
    pipe_name = f'vcak.{flange_number}'
    
    L3 = line.get_s_position(f'vcak.{flange_number}.b_aper') - line.get_s_position(f'vcak.{flange_number}.a_aper') - L1 - L2
    if L3 < 0:
        raise ValueError(f'Pipe {pipe_name} too short to fit all apertures, please adjust L1 and L2')
    
    pipe_data[pipe_name] = {'apertures': [xt.LimitEllipse(a=r1, b=r1, shift_x=-5.3e-3), 
                                  xt.LimitEllipse(a=r2, b=r2, shift_x=-5.3e-3),
                                  xt.LimitEllipse(a=r3, b=r3)],
                            'lengths': [L1, L2, L3],
                            #'edge_names': [f'{pipe_name}.a_aper', f'{pipe_name}.b_aper'],
                            'edge_positions': [line.get_s_position(f'vcak.{flange_number}.a_aper'), line.get_s_position(f'vcak.{flange_number}.b_aper')]
                            }
    

pipe_data = find_pipe_edges(line, pipe_data)

slicing_strategies = get_slicing_strategies(line, pipe_data)
line.slice_thick_elements(slicing_strategies)

Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           
The line already has an associated tracker


Slicing line:   0%|          | 0/32246 [00:00<?, ?it/s]

{'sps$start': ['sps$start'],
 'begi.10010': ['begi.10010'],
 'qf.10010': ['qf.10010'],
 'drift_0..0': ['drift_0..0'],
 'veqf.10010.b_aper': ['veqf.10010.b_aper'],
 'drift_0..1': ['drift_0..1'],
 'veba.10030.a_aper': ['veba.10030.a_aper'],
 'drift_0..2': ['drift_0..2'],
 'mba.10030_entry': ['mba.10030_entry'],
 'mba.10030..entry_map': ['mba.10030..entry_map'],
 'mba.10030..0_aper2': ['mba.10030..0_aper2'],
 'mba.10030..0': ['mba.10030..0'],
 'mba.10030..1_aper2': ['mba.10030..1_aper2'],
 'mba.10030..1': ['mba.10030..1'],
 'mba.10030..2_aper2': ['mba.10030..2_aper2'],
 'mba.10030..2': ['mba.10030..2'],
 'mba.10030..3_aper2': ['mba.10030..3_aper2'],
 'mba.10030..3': ['mba.10030..3'],
 'mba.10030..4_aper2': ['mba.10030..4_aper2'],
 'mba.10030..4': ['mba.10030..4'],
 'mba.10030..5_aper2': ['mba.10030..5_aper2'],
 'mba.10030..5': ['mba.10030..5'],
 'mba.10030..6_aper2': ['mba.10030..6_aper2'],
 'mba.10030..6': ['mba.10030..6'],
 'mba.10030..7_aper2': ['mba.10030..7_aper2'],
 'mba.10030..7': 

In [97]:
tt = line.get_table()
pipe_name = 'vcak.13501'
pipe_start, pipe_end = pipe_data[pipe_name]['edge_names']

collimators = []
collimator_names = []
positions = []
l_acc = 0
insertions = []
for i, (l, aper) in enumerate(zip(pipe_data[pipe_name]['lengths'], pipe_data[pipe_name]['apertures'])):
    names = list(tt.rows[(tt.s>=line.get_s_position(pipe_start)+l_acc) & (tt.s<=line.get_s_position(pipe_start)+l_acc+l)].name)
    elements = list(tt.rows[(tt.s>=line.get_s_position(pipe_start)+l_acc) & (tt.s<=line.get_s_position(pipe_start)+l_acc+l)].element_type)
    
    print(names, elements)
    #Removing edge apertures of the list to consider
    if pipe_name in names[0]:
        names.pop(0)
        elements.pop(0)
    elif pipe_name in names[-1]:
        names.pop(-1)
        elements.pop(-1)

    #Defining jaws from aperture
    if aper.__class__.__name__ == 'LimitEllipse':
        jaw_L = aper.a - aper.shift_x
        jaw_R = -aper.a - aper.shift_x
    else:
        jaw_L = aper.max_x - aper.shift_x
        jaw_R = aper.min_x - aper.shift_x
    
    prev_nn = None
    counter = 0
    for nn, ee in zip(names, elements):
        if ee.startswith('Drift') or ee.startswith('Marker') or ee.startswith('Limit'):
            continue
        else:
            #Active element in the way
            if prev_nn is None:
                collimators.append(xc.EverestCollimator(length=line.get_s_position(nn) - line.get_s_position(pipe_start) - l_acc, material=xc.materials.Glidcop, jaw_L=jaw_L, jaw_R=jaw_R))
                positions.append(line.get_s_position(pipe_start))
                collimator_names.append(f'coll_{pipe_name}_{i}..{counter}')
                line.env.elements[f'coll_{pipe_name}_{i}..{counter}'] = xc.EverestCollimator(length=line.get_s_position(nn) - line.get_s_position(pipe_start) - l_acc, material=xc.materials.Glidcop, jaw_L=jaw_L, jaw_R=jaw_R)
                #insertions.append(line.env.place(f'coll_{pipe_name}_{i}..{counter}', at=f'{line.get_s_position(pipe_start)+l_acc}@end', anchor='start'))
                insertions.append(line.env.place(f'coll_{pipe_name}_{i}..{counter}', at=tt.rows[f'{prev_nn}>>1'].name, anchor='start'))
                prev_nn = nn
                counter += 1
            else:
                print(prev_nn,nn)
                # collimators.append(xc.EverestCollimator(length=line.get_s_position(nn) - line.get_s_position(prev_nn)-l_acc, material=xc.materials.Glidcop, jaw_L=jaw_L, jaw_R=jaw_R))
                # positions.append(line.get_s_position(prev_nn))
                # prev_nn = nn
                line.env.elements[f'coll_{pipe_name}_{i}..{counter}'] = xc.EverestCollimator(length=line.get_s_position(nn) - line.get_s_position(prev_nn)-l_acc, material=xc.materials.Glidcop, jaw_L=jaw_L, jaw_R=jaw_R)
                #insertions.append(line.env.place(f'coll_{pipe_name}_{i}..{counter}', at=f'{line.get_s_position(prev_nn)}@end', anchor='start'))
                insertions.append(line.env.place(f'coll_{pipe_name}_{i}..{counter}', at=tt.rows[f'{prev_nn}>>1'].name, anchor='start'))
                prev_nn = nn
                counter += 1
    l_acc += l
line.insert(insertions, s_tol=1e-6)

['vcak.13501.a_aper', 'drift_299..2'] ['LimitEllipse', 'DriftSlice']
['lod.13502_entry', 'lod.13502..entry_map', 'drift_lod.13502..0'] ['Marker', 'ThinSliceOctupoleEntry', 'DriftSliceOctupole']
['lod.13502..0', 'drift_lod.13502..1', 'lod.13502..1', 'drift_lod.13502..2', 'lod.13502..2', 'drift_lod.13502..3', 'lod.13502..3', 'drift_lod.13502..4', 'lod.13502..4', 'drift_lod.13502..5', 'lod.13502..exit_map', 'lod.13502_exit', 'drift_300..0'] ['ThinSliceOctupole', 'DriftSliceOctupole', 'ThinSliceOctupole', 'DriftSliceOctupole', 'ThinSliceOctupole', 'DriftSliceOctupole', 'ThinSliceOctupole', 'DriftSliceOctupole', 'ThinSliceOctupole', 'DriftSliceOctupole', 'ThinSliceOctupoleExit', 'Marker', 'DriftSlice']
lod.13502..0 lod.13502..1
lod.13502..1 lod.13502..2
lod.13502..2 lod.13502..3
lod.13502..3 lod.13502..4
lod.13502..4 lod.13502..exit_map


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
line.env.elements['vcak.13501.coll..0'] = xc.EverestCollimator(length=L1, material=xc.materials.Glidcop, jaw=r1-5.3e-3)
line.insert(env.place('vcak.13501.coll..0', at=f'vcak.13501.a_aper@end', anchor='start'), s_tol=1e-6)

In [147]:
tt = line.get_table()
pipe_name = 'vcak.13501'
pipe_start, pipe_end = pipe_data[pipe_name]['edge_names']

collimators = []
collimator_names = []
positions = []
l_acc = 0
insertions = []
for i, (l, aper) in enumerate(zip(pipe_data[pipe_name]['lengths'], pipe_data[pipe_name]['apertures'])):
    print(i, l, l_acc)
    names = list(tt.rows[(tt.s>=line.get_s_position(pipe_start)+l_acc) & (tt.s<=line.get_s_position(pipe_start)+l_acc+l)].name)
    elements = list(tt.rows[(tt.s>=line.get_s_position(pipe_start)+l_acc) & (tt.s<=line.get_s_position(pipe_start)+l_acc+l)].element_type)
    
    print(names, elements)
    #Removing edge apertures of the list to consider
    if pipe_name in names[0]:
        names.pop(0)
        elements.pop(0)
    elif pipe_name in names[-1]:
        names.pop(-1)
        elements.pop(-1)

    #Defining jaws from aperture
    if aper.__class__.__name__ == 'LimitEllipse':
        jaw_L = aper.a - aper.shift_x
        jaw_R = -aper.a - aper.shift_x
    else:
        jaw_L = aper.max_x - aper.shift_x
        jaw_R = aper.min_x - aper.shift_x
    
    prev_nn = None
    counter = 0
    DriftEncountered = False
    ActiveElementEncountered = False
    for j, (nn, ee) in enumerate(zip(names, elements)):
        if ee.startswith('Marker') or ee.startswith('Limit'):
            continue
        elif ee.startswith('Drift') and not DriftEncountered:
            DriftEncountered = True
            start_coll = line.get_s_position(nn)
            start_drift = nn
            print(start_drift)
        else:
            #active element encountered
            line.env.elements[f'coll_{pipe_name}_{i}..{counter}'] = xc.EverestCollimator(length=line.get_s_position(nn) - start_coll, material=xc.materials.Glidcop, jaw_L=jaw_L, jaw_R=jaw_R)
            print(line.get_s_position(nn) - start_coll)
            # insertions.append(line.env.place(f'coll_{pipe_name}_{i}..{counter}', at=f'{start_coll}@start'))
            insertions.append(line.env.place(f'coll_{pipe_name}_{i}..{counter}', at=f'{start_drift}@start'))
            if hasattr(line.env.elements[start_drift],'length'):
                print(f'coll_{pipe_name}_{i}', start_drift, line.env.elements[start_drift].length, line.get_s_position(nn) - start_coll)
            else:
                print(f'coll_{pipe_name}_{i}', start_drift, line.env.elements[start_drift]._parent.length * line.env.elements[start_drift].weight, line.get_s_position(nn) - start_coll)
            counter += 1
            DriftEncountered = False
            ActiveElementEncountered = True
    if not ActiveElementEncountered:
        line.env.elements[f'coll_{pipe_name}_{i}'] = xc.EverestCollimator(length=l, material=xc.materials.Glidcop, jaw_L=jaw_L, jaw_R=jaw_R)
        # insertions.append(line.env.place(f'coll_{pipe_name}_{i}', at=f'{start_coll}@start'))
        insertions.append(line.env.place(f'coll_{pipe_name}_{i}', at=f'{start_drift}@start'))
        if hasattr(line.env.elements[start_drift],'length'):
            print(f'coll_{pipe_name}_{i}', start_drift, line.env.elements[start_drift].length, l)
        else:
            print(f'coll_{pipe_name}_{i}', start_drift, line.env.elements[start_drift]._parent.length, l)
    l_acc += l
line.insert(insertions, s_tol=1e-6)

0 0.034 0
['vcak.13501.a_aper', 'drift_299..2'] ['LimitEllipse', 'DriftSlice']
drift_299..2
coll_vcak.13501_0 drift_299..2 0.29849069860983946 0.034
1 0.07 0.034
['lod.13502_entry', 'lod.13502..entry_map', 'drift_lod.13502..0'] ['Marker', 'ThinSliceOctupoleEntry', 'DriftSliceOctupole']
0.10219999999799256
coll_vcak.13501_1 drift_299..2 0.10219999999799255 0.10219999999799256
drift_lod.13502..0
2 0.7547999999997463 0.10400000000000001
['lod.13502..0', 'drift_lod.13502..1', 'lod.13502..1', 'drift_lod.13502..2', 'lod.13502..2', 'drift_lod.13502..3', 'lod.13502..3', 'drift_lod.13502..4', 'lod.13502..4', 'drift_lod.13502..5', 'lod.13502..exit_map', 'lod.13502_exit', 'drift_300..0'] ['ThinSliceOctupole', 'DriftSliceOctupole', 'ThinSliceOctupole', 'DriftSliceOctupole', 'ThinSliceOctupole', 'DriftSliceOctupole', 'ThinSliceOctupole', 'DriftSliceOctupole', 'ThinSliceOctupole', 'DriftSliceOctupole', 'ThinSliceOctupoleExit', 'Marker', 'DriftSlice']
0.0564166666665642
coll_vcak.13501_2 drift_lod.13

Slicing line:   0%|          | 0/32330 [00:00<?, ?it/s]

AssertionError: Negative drift length: -0.06809999999904903, upstream of coll_vcak.13501_1..0

In [148]:
line.get_s_position('lod.13502_entry') - line.get_s_position('vcak.13501.a_aper')

np.float64(0.10219999999799256)

In [134]:
tt.rows['vcak.13501.a_aper':'vcak.13501.b_aper']

Table: 19 rows, 11 cols
name                             s element_type           isthick isreplica parent_name ...
vcak.13501.a_aper          1117.77 LimitEllipse             False     False None       
drift_299..2               1117.77 DriftSlice                True     False drift_299  
lod.13502_entry            1117.88 Marker                   False     False None       
lod.13502..entry_map       1117.88 ThinSliceOctupoleEntry   False     False lod.13502  
drift_lod.13502..0         1117.88 DriftSliceOctupole        True     False lod.13502  
lod.13502..0               1117.93 ThinSliceOctupole        False     False lod.13502  
drift_lod.13502..1         1117.93 DriftSliceOctupole        True     False lod.13502  
lod.13502..1               1118.07 ThinSliceOctupole        False     False lod.13502  
drift_lod.13502..2         1118.07 DriftSliceOctupole        True     False lod.13502  
lod.13502..2               1118.21 ThinSliceOctupole        False     False lod.13502  
drif

In [145]:
line.env.elements['drift_299..2']._parent.length * line.env.elements['drift_299..2'].weight

np.float64(0.10219999999799255)

In [138]:
L1

0.034

In [144]:
tt.rows['lod.13502_entry'].s[0] - tt.rows['drift_299..2'].s[0]

np.float64(0.10219999999799256)

In [141]:
line.get_s_position('lod.13502_entry') - line.get_s_position('drift_299..2')

AssertionError: 